In [4]:
from imp import load_module
import os
from flask import Flask, flash, jsonify, redirect, request, send_from_directory
from werkzeug.utils import secure_filename
from ultralytics import YOLO
from PIL import Image, ImageDraw
from flask_cors import CORS
import time
from flask import Flask, request, jsonify
import torch
from torchvision import transforms
from PIL import Image




UPLOAD_FOLDER = './store'
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'}

app = Flask(__name__)

CORS(app, resources={r"/*": {"origins": "*"}})

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

model = YOLO('./best.pt')

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    print('Received a request')
    if request.method == 'POST':
       
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
       
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            results = model.predict(os.path.join(app.config['UPLOAD_FOLDER'], filename), conf = 0.4, save_dir = "./output/")
            names = model.names
            store = set()
            for r in results[0]:
                for c in r.boxes.cls:
                    store.add(names[int(c)])
            names_list = list(store)
            
            
            image_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            image = Image.open(image_path)
            classes = {0:"Sini", 1:"Jebena"}
            # Draw bounding boxes and labels of detections
            draw = ImageDraw.Draw(image)

            for box_info in results[0].boxes:
                
                x_min, y_min, x_max, y_max = box_info.xyxy.squeeze().tolist()  
                class_id = int(box_info.cls.item())  
                confidence = float(box_info.conf.item()) 
               
                box_color = 'red' if classes[class_id] == 'Jebena' else 'green'
                label = f'{classes[class_id]}'

               
                draw.rectangle([(x_min, y_min), (x_max, y_max)], outline=box_color, width = 6)
                # font =  ImageFont.truetype("arial.ttf", 15)
                draw.text((x_min, y_min), label, fill=box_color, )

            
            # image.show()  
            image.save(os.path.join("/home/geleta/Desktop/PAI_ASSIGNMENT/pai_project/assets", filename))
            
            # time.sleep(5)
            return jsonify({'names': names_list, 'image_path': filename})

    return '''
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Upload>
    </form>
    '''

@app.route('/get_image/<filename>', methods=['GET'])
def get_image(filename):
    return send_from_directory('/home/geleta/Desktop/PAI_ASSIGNMENT/pai_project/assets', filename)




if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.212.126:5000
Press CTRL+C to quit


Received a request

image 1/1 /home/geleta/Desktop/Ethiopic-Object-Detection-App/Backend/store/scaled_1000001001.jpg_rotated_360.jpg: 448x640 1 Jebena, 241.9ms
Speed: 7.0ms preprocess, 241.9ms inference, 1.6ms postprocess per image at shape (1, 3, 448, 640)


192.168.212.157 - - [31/Jan/2024 21:20:16] "POST / HTTP/1.1" 200 -
192.168.212.157 - - [31/Jan/2024 21:20:16] "GET /get_image/scaled_1000001001.jpg_rotated_360.jpg HTTP/1.1" 200 -


Received a request

image 1/1 /home/geleta/Desktop/Ethiopic-Object-Detection-App/Backend/store/scaled_1000001000.jpg_rotated_360.jpg: 448x640 1 Sini, 7 Jebenas, 171.1ms
Speed: 4.9ms preprocess, 171.1ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)


...

192.168.212.157 - - [31/Jan/2024 21:24:59] "POST / HTTP/1.1" 200 -
192.168.212.157 - - [31/Jan/2024 21:24:59] "GET /get_image/scaled_c293cbf5-1e41-4615-be96-33f38342dfe41826651652236339726.jpg_rotated_360.jpg HTTP/1.1" 200 -


Received a request

image 1/1 /home/geleta/Desktop/Ethiopic-Object-Detection-App/Backend/store/scaled_d622bc91-9c21-4fa6-8e28-25bc0ea5f7751607038023041872590.jpg_rotated_360.jpg: 288x640 3 Jebenas, 77.5ms
Speed: 2.4ms preprocess, 77.5ms inference, 0.9ms postprocess per image at shape (1, 3, 288, 640)


192.168.212.157 - - [31/Jan/2024 21:25:19] "POST / HTTP/1.1" 200 -
192.168.212.157 - - [31/Jan/2024 21:25:19] "GET /get_image/scaled_d622bc91-9c21-4fa6-8e28-25bc0ea5f7751607038023041872590.jpg_rotated_360.jpg HTTP/1.1" 200 -


Received a request

image 1/1 /home/geleta/Desktop/Ethiopic-Object-Detection-App/Backend/store/scaled_f7ca302f-c503-4798-acf3-9c7c4fc090a25475034607360072700.jpg_rotated_360.jpg: 288x640 1 Jebena, 84.4ms
Speed: 2.2ms preprocess, 84.4ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 640)


192.168.212.157 - - [31/Jan/2024 21:25:44] "POST / HTTP/1.1" 200 -
192.168.212.157 - - [31/Jan/2024 21:25:44] "GET /get_image/scaled_f7ca302f-c503-4798-acf3-9c7c4fc090a25475034607360072700.jpg_rotated_360.jpg HTTP/1.1" 200 -
